<a id="top"></a>
# Compute Items Bounding Box
First:
* [Load Parameters and Items](#load)

Then choose one operation:
* [Compute and Save](#compute)
* [Statistics](#statistics)
* [Plot Item Files](#plot)
* [Computing Test](#test)

## Load Parameters and Items<a id="load"></a>

In [ ]:
import json
from utils import load_items

with open('parameters.json', 'r') as infile:
    params = json.load(infile)

RESIZE_X = params['resize']['x']
RESIZE_Y = params['resize']['y']
ITEM_FOLDER = params['item_folder']

items = load_items(ITEM_FOLDER)

## Compute and Save<a id="compute"></a>

In [ ]:
import cv2, glob, json
from utils import imread_gray

def worker(item):
    folder = ITEM_FOLDER + '/' + item + '/'
    files = glob.glob(folder + '*_mask.pgm')
    for filename in files:
        mask = imread_gray(filename)
        if not mask is None:
            cnt, _ = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
            cnt = sorted(cnt, key=lambda x:cv2.contourArea(x), reverse=True)
            x,y,w,h = cv2.boundingRect(cnt[0])
            bbox = {'x':x,'y':y,'w':w,'h':h}
            with open(filename[:-9] + '_bbox.json', 'w') as outfile:
                json.dump(bbox, outfile)

In [ ]:
%%time
from multiprocessing import Pool

print('Computing bounding boxes of images')
print('* resized to %d x %d' % (RESIZE_X,RESIZE_Y))
pool_size = 6
pool = Pool(pool_size)
result = []
for item in items:
    result.append( pool.apply_async(worker, (item,)) )
pool.close()
pool.join()
for r in result:
     r.get()

## Statistics<a id="statistics"></a>

In [ ]:
import glob, json

item_view = []
area = []
for item in items:
    folder = ITEM_FOLDER + '/' + item + '/'
    files = glob.glob(folder + '*_bbox.json')
    for filename in files:
        with open(filename, 'r') as infile:
            bbox = json.load(infile)
        x = bbox['x']
        y = bbox['y']
        w = bbox['w']
        h = bbox['h']
        item_view.append(filename)
        area.append(w*h)

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

plt.hist(area,bins=30);

In [ ]:
[(a, str(iv.split('/')[-1][:-10])) for a, iv in sorted(zip(area,item_view), reverse=True) if a>230000]

In [ ]:
[(a, str(iv.split('/')[-1][:-10])) for a, iv in sorted(zip(area,item_view), reverse=True) if a<50000]

## Plot File<a id="plot"></a>

In [ ]:
import cv2
from utils import imread_rgb
from matplotlib import pyplot as plt
%matplotlib inline

def load_and_plot(item,view):
    filename = ITEM_FOLDER + '/' + item + '/' + item + '_' + view + '.png'
    image_RGB = imread_rgb(filename)
    if not image_RGB is None:
        image_RGB = cv2.resize(image_RGB,(RESIZE_X,RESIZE_Y))
        with open(filename[:-4] + '_bbox.json', 'r') as infile:
            bbox = json.load(infile)
        x = bbox['x']
        y = bbox['y']
        w = bbox['w']
        h = bbox['h']
        image_plot = image_RGB.copy()
        cv2.rectangle(image_plot,(x,y),(x+w,y+h),(0,255,0),2)
        plt.subplot(121), plt.imshow(image_plot);
        plt.subplot(122), plt.imshow(image_RGB[y:y+h,x:x+w]), plt.axis('off');

In [ ]:
from ipywidgets import interact
views = ['top_01','top-side_01','top-side_02','bottom_01','bottom-side_01','bottom-side_02']
interact(load_and_plot,item=items,view=views);

### Plot All Items

In [ ]:
for item in items:
    for view in views:
        print(item + '_' + view)
        load_and_plot(item,view)
        plt.show()

## Computing Test<a id="test"></a>

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
import cv2, numpy as np
from ipywidgets import interact
from utils import imread_rgb, imread_gray

def compute_and_plot(item,view):
    prefix = ITEM_FOLDER + '/' + item + '/' + item + '_' + view
    filename = prefix + '.png'
    image_RGB = imread_rgb(filename)
    if not image_RGB is None:
        image_RGB = cv2.resize(image_RGB,(RESIZE_X,RESIZE_Y))
        filename = prefix + '_mask.pgm'
        mask = imread_gray(filename)
        if not mask is None:
            cnt, _ = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
            cnt = sorted(cnt, key=lambda x:cv2.contourArea(x), reverse=True)
            x,y,w,h = cv2.boundingRect(cnt[0])
            image_plot = image_RGB.copy()
            cv2.rectangle(image_plot,(x,y),(x+w,y+h),(0,255,0),2)
            plt.subplot(121), plt.imshow(image_plot);
            plt.subplot(122), plt.imshow(image_RGB[y:y+h,x:x+w]), plt.axis('off');

In [ ]:
views = ['top_01','top-side_01','top-side_02','bottom_01','bottom-side_01','bottom-side_02']
interact(compute_and_plot,item=items,view=views);

### Compute and Plot All Items

In [ ]:
for item in items:
    for view in views:
        print(item + '_' + view)
        compute_and_plot(item,view)
        plt.show()

[Top](#top)